# Project 3, Part 2, Queries on the stations, lines, and travel times tables for the BART system that we will use later to create the graph

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering

Students in the group:
* Aris Chalini
* Jack Galvin
* Matt Lauritzen

Year: 2022

Semester: Spring

Section: 09


# Included Modules and Packages

Code cell containing your includes for modules and packages

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [1]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  

Remember you can freely use any code from the labs. You do not need to cite code from the labs.

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

# 3.2.1 Query the list of stations

Show the station from the stations table

Sort by station

The query should return 50 rows

Display the results in a Pandas data frame

In [5]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station
from stations
order by station

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station
0,12th Street
1,16th Street Mission
2,19th Street
3,24th Street Mission
4,Antioch
5,Ashby
6,Balboa Park
7,Bay Fair
8,Berryessa
9,Castro Valley


# 3.2.2 Query the lines served by the West Oakland station

The West Oakland station should serve the 4 following lines:

|station|line|
|-------|----|
|West Oakland|blue|
|West Oakland|green|
|West Oakland|red|
|West Oakland|yellow|

Show the station and line from the lines table where the station is West Oakland

Sort by station, line

Display the results in a Pandas data frame

In [6]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station, line
from lines
where station = 'West Oakland'
order by 1, 2

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,line
0,West Oakland,blue
1,West Oakland,green
2,West Oakland,red
3,West Oakland,yellow


# 3.2.3 Query the list of stations and the lines they serve

Same query as the last one, but for all stations, not just limited to West Oakland

There should be 114 row returned

Display the results in a Pandas data frame

In [7]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station, line
from lines
order by 1, 2

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,line
0,12th Street,orange
1,12th Street,red
2,12th Street,yellow
3,16th Street Mission,blue
4,16th Street Mission,green
...,...,...
109,West Dublin,blue
110,West Oakland,blue
111,West Oakland,green
112,West Oakland,red


# 3.2.4 Query the list of all possible line transfers at the West Oakland station and the transfer times

A transfer is when two or more lines pass through a station and the passenger gets off one line and gets on another line.

At West Oakland, there are 12 possible transfers:

|station|from_line|to_line|transfer_time|
|-------|---------|-------|-------------|
|West Oakland|blue|green|283|
|West Oakland|blue|red|283|
|West Oakland|blue|yellow|283|
|West Oakland|green|blue|283|
|West Oakland|green|red|283|
|West Oakland|green|yellow|283|
|West Oakland|red|blue|283|
|West Oakland|red|green|283|
|West Oakland|red|yellow|283|
|West Oakland|yellow|blue|283|
|West Oakland|yellow|green|283|
|West Oakland|yellow|red|283|

Show the station, from line, to line, transfer time in seconds where the station is West Oakland

Sort by station, from line, to line

Hints:

Join the lines table to itself where the station is the same, but the lines are different

Then join to the stations table to pull the transfer time

Display the results in a Pandas data frame

In [8]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines as a
join lines as b
    on a.station = b.station
join stations as s
    on a.station = s.station
where a.station = 'West Oakland'
    and a.line != b.line
order by 1, 2, 3

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,from_line,to_line,transfer_time
0,West Oakland,blue,green,283
1,West Oakland,blue,red,283
2,West Oakland,blue,yellow,283
3,West Oakland,green,blue,283
4,West Oakland,green,red,283
5,West Oakland,green,yellow,283
6,West Oakland,red,blue,283
7,West Oakland,red,green,283
8,West Oakland,red,yellow,283
9,West Oakland,yellow,blue,283


# 3.2.5 Query the list of all possible line transfers and the transfer times

Same query as the last query, but all stations, not just limited to West Oakland

The query should return 208 rows

Display the results in a Pandas data frame

In [9]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines as a
join lines as b
    on a.station = b.station
join stations as s
    on a.station = s.station
where a.line != b.line
order by 1, 2, 3

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,from_line,to_line,transfer_time
0,12th Street,orange,red,282
1,12th Street,orange,yellow,282
2,12th Street,red,orange,282
3,12th Street,red,yellow,282
4,12th Street,yellow,orange,282
...,...,...,...,...
203,West Oakland,red,green,283
204,West Oakland,red,yellow,283
205,West Oakland,yellow,blue,283
206,West Oakland,yellow,green,283


# 3.2.6 Query the list of all segments between the West Oakland station and its adjoining stations


West Oakland has the following 8 segments between it and the stations it adjoins:

|line|from station|to station|travel time in seconds|
|----|------------|----------|----------------------|
|blue|Lake Merritt|West Oakland|360|
|blue|West Oakland|Embarcadero|420|
|green|Lake Merritt|West Oakland|360|
|green|West Oakland|Embarcadero|420|
|red|12th Street|West Oakland|300|
|red|West Oakland|Embarcadero|420|
|yellow|12th Street|West Oakland|300|
|yellow|West Oakland|Embarcadero|420|

Show the line, from station, to station, travel time in seconds

Sort by line, from station, to station

Note: Since the lines table is only showing one way for each line, we will only get one way in this query.  This is fine.  Later when we build the graph, we will simply create the relationships going both ways.

Hints:

Join lines to itself where the line is the same and the sequence is equal to the sequence + 1

Then join to the travel_times table matching stations and pulling the travel time (note that the stations may be in either order in the travel_times table)

Display the results in a Pandas data frame

In [10]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.line, 
    a.station as "from station", 
    b.station as "to station", 
    t.travel_time as "travel time in seconds"
from lines as a
join lines as b
    on a.line = b.line 
        and a.sequence = b.sequence - 1
join travel_times as t
    on (a.station = t.station_1 and b.station = station_2)
        or (a.station = t.station_2 and b.station = station_1)
Where a.station = 'West Oakland' 
    or b.station = 'West Oakland'
order by 1, 2, 3


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,from station,to station,travel time in seconds
0,blue,Lake Merritt,West Oakland,360
1,blue,West Oakland,Embarcadero,420
2,green,Lake Merritt,West Oakland,360
3,green,West Oakland,Embarcadero,420
4,red,12th Street,West Oakland,300
5,red,West Oakland,Embarcadero,420
6,yellow,12th Street,West Oakland,300
7,yellow,West Oakland,Embarcadero,420


# 3.2.7 Query the list of all segments between each station and its adjoining stations

Same query as the last query, but for all stations, not limited to West Oakland.

The query should return 108 rows

Display the results in a Pandas data frame

In [11]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.line, 
    a.station as "from station", 
    b.station as "to station", 
    t.travel_time as "travel time in seconds"
from lines as a
join lines as b
    on a.line = b.line 
        and a.sequence = b.sequence - 1
join travel_times as t
    on (a.station = t.station_1 and b.station = station_2)
        or (a.station = t.station_2 and b.station = station_1)
order by 1, 2, 3

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,from station,to station,travel time in seconds
0,blue,16th Street Mission,24th Street Mission,120
1,blue,24th Street Mission,Glen Park,180
2,blue,Balboa Park,Daly City,240
3,blue,Bay Fair,San Leandro,240
4,blue,Castro Valley,Bay Fair,240
...,...,...,...,...
103,yellow,Rockridge,MacArthur,240
104,yellow,San Bruno,SFO,240
105,yellow,South San Francisco,San Bruno,240
106,yellow,Walnut Creek,Lafayette,300
